In [1]:
import numpy as np

In [2]:
def gini_impurity(y: np.ndarray) -> float:
    # G = 1 - sum(pk)^2
    p1 = sum(y) / len(y)
    p0 = 1 - p1
    return 1 - p0**2 - p1**2


def entropy(y: np.ndarray) -> float:
    # entropy = - sum pk*log2(pk)
    p1 = sum(y) / len(y)
    p0 = 1 - p1
    if p0 ==0:
        return  - p1*np.log2(p1)
    elif p1 == 0:
        return  - p0*np.log2(p0)
    return - p0*np.log2(p0) - p1*np.log2(p1)


def inform_gain(X: np.ndarray, y: np.ndarray, threshold: float, criteria_func=entropy) -> float:
    #IG(Q) = S0 - sum Ni*Si/N
    S0 = criteria_func(y)
    mask1 = (X <= threshold)
    mask2 = (X > threshold)
    S1 = criteria_func(y[mask1])
    if sum(mask2) ==0:
        S2 = 0
    else:
        S2 = criteria_func(y[mask2])
    N1 = sum(mask1)
    N2 = sum(mask2)
    N = len(X)
    return np.round(S0 - N1*S1/N - N2*S2/N,4)

def get_best_threshold(X: np.ndarray, y: np.ndarray, criteria_func=entropy) -> (float, float):
    assert X.ndim == 1
    assert y.ndim == 1
    best_threshold = 0
    best_score = 0
    for val in np.unique(X):
        IG = inform_gain(X,y,val,criteria_func)
        if IG >= best_score:
            best_threshold = val
            best_score = IG
    return best_threshold, best_score

def find_best_split(X, y, criteria_func=entropy):
    assert X.ndim == 2
    assert y.ndim == 1
    best_feature = 0
    best_score = 0
    best_threshold = 0
    n,m = X.shape
    ### делаем цикл по всем фичам в X и находим для каждой лучший threshold и score.
    ### Потом сравниваем с наилучшим (best)
    for j in range(m):
        thr, score = get_best_threshold(X[:,j], y, criteria_func)

        if score >= best_score:
            best_score = score
            best_feature = j
            best_threshold = thr

    return best_feature, best_threshold, best_score

class MyDecisionTreeClassifier():
    def __init__(self, max_depth=4, criterion='entropy'):
        self.max_depth = max_depth
        self.criterion = criterion # 'entropy' or 'gini'
        self.tree = {}
        self._criteria_func = {
            'gini': gini_impurity,
            'entropy': entropy
        }
        self.count_of_features = 0

    def _build_tree(self, X, y, depth=0):

        split_feature, split_val, score = find_best_split(X,y,self._criteria_func[self.criterion])

        # критерий остановки: если мы достигли нужной глубины или если скор оказался равен нулю
        # технически со скором 0 нужно проверить потенциальную разбиваемость на след уровнях, но это сложно, так что пока так закостылим:)
        if depth == self.max_depth or score == 0:
            p1 = sum(y) / len(y)
            p0 = 1 - p1
            return {'leaf': True,
                    'proba': [p0, p1]
                   }

        left_ids = X[:, split_feature] <= split_val
        right_ids = X[:, split_feature] > split_val

        left_tree  =  self._build_tree(X[left_ids], y[left_ids], depth + 1)
        right_tree = self._build_tree(X[right_ids], y[right_ids], depth + 1)
        return {'leaf': False,
                'val': {'split_feature': split_feature,
                        'split_val': split_val,
                        'score': score},
                'left': left_tree,
                'right': right_tree}

    def _predict_proba_obj(self, obj: np.array):
        assert obj.ndim == 1

        node = self.tree

        ### необходимо спустится в нужную листовую ноду
        while not node['leaf']:
            if obj[node['val']['split_feature']] <= node['val']['split_val']:
                node = node['left']
            else:
                node = node['right']

        assert node['leaf'] == True
        return node['proba']

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.count_of_features = X.shape[1]
        self.tree = self._build_tree(X, y, depth=0)
        return self

    def predict_proba(self, X: np.ndarray):
        assert X.shape[1] == self.count_of_features
        proba = []
        for row in X:
            proba.append(self._predict_proba_obj(row))
        return np.array(proba)

    def predict(self, X: np.ndarray): # получаем
        assert X.shape[1] == self.count_of_features
        proba = self.predict_proba(X)
        return np.array(list(np.argmax(proba, axis=1)))
